In [4]:
# use su3, before quantization
import random


graphname = "dring"
routingname = "su3"
numsw = 80
numport = 64
failpct = 0.02
seed = 0
random.seed(seed)

homedir = "/home/annzhou"
netpathfile = f"{homedir}/DRing/src/emp/datacentre/netpathfiles/netpath_{routingname}_{graphname}.txt"
graphfile = f"{homedir}/DRing/src/emp/datacentre/graphfiles/ring_supergraph/double_ring/instance1_{numsw}_{numport}.edgelist"

# read netpathfile
netpath = list()
for i in range(numsw):
    netpath.append(list())
    for j in range(numsw):
        netpath[i].append(list())
with open(netpathfile,'r') as f:
    lines = f.readlines()
    # produce
    fromsw = 0
    tosw = 0
    for line in lines:
        if "->" not in line:
            tokens = line.split()
            fromsw = int(tokens[0])
            tosw = int(tokens[1])
        else:
            path = [fromsw]
            tokens = line.split()
            for token in tokens:
                hops = token.split("->")
                path.append(int(hops[1]))
            netpath[fromsw][tosw].append(path)

    # check
    for line in lines:
        if "->" not in line:
            tokens = line.split()
            fromsw = int(tokens[0])
            tosw = int(tokens[1])
            numpaths = int(tokens[2])
            if len(netpath[fromsw][tosw])!=numpaths:
                print(f"ERROR: netpath is wrong, fromsw={fromsw}, tosw={tosw}, numpaths from file={numpaths}, numpaths from array={len(netpath[fromsw][tosw])}")

# read graphfile
link = list()
numlink = 0
for i in range(numsw):
    link.append(list())
    for j in range(numsw):
        link[i].append(0)
with open(graphfile,'r') as f:
    lines = f.readlines()
    for line in lines:
        tokens = line.split("->")
        fromsw = int(tokens[0])
        tosw = int(tokens[1])
        link[fromsw][tosw] = 1
        link[tosw][fromsw] = 1
        numlink += 2

faillinklist = list()
numfaillink = int(numlink*failpct)
while len(faillinklist) < numfaillink:
    r = random.randint(0,numlink-1)
    if r not in faillinklist:
        faillinklist.append(r)

isfaildict = dict()
ilink = 0
for i in range(numsw):
    for j in range(numsw):
        if link[i][j] == 1:
            linkstr = f"{i},{j}"
            if ilink in faillinklist:
                isfaildict[linkstr] = 1
            else:
                isfaildict[linkstr] = 0
            ilink += 1
if ilink != numlink:
    print(f"ERROR: ilink!=numlink, ilink={ilink}, numlink={numlink}")

mynumpathlist = list()
for i in range(numsw):
    for j in range(numsw):
        if i==j: continue
        if len(netpath[i][j]) == 0:
            mynumpathlist.append(0)
            continue
        numpath = 0
        for path in netpath[i][j]:
            ispathvalid = True
            for ihop,hop in enumerate(path):
                if ihop == 0: continue
                if isfaildict[f"{path[ihop-1]},{hop}"] == 1:
                    ispathvalid = False
                    break
            if ispathvalid:
                numpath+=1
        mynumpathlist.append(numpath)
print(sum(mynumpathlist)/len(mynumpathlist))

231.28354430379747
